In [2]:
import numpy as np
from scipy.io.wavfile import write
from scipy.signal import butter, lfilter

In [3]:
def generate_white_noise(duration=1, sampling_rate=44100, sample_depth=16,
                         quantization_format='int', filename='white_noise.wav',
                         cutoff_freq=None, filter_order=5):
    # Generate white noise
    samples = np.random.uniform(low=-1.0, high=1.0, size=sampling_rate * duration)

    # Apply low-pass filter if cutoff frequency is specified
    if cutoff_freq is not None:
        nyquist = 0.5 * sampling_rate
        normalized_cutoff = cutoff_freq / nyquist
        b, a = butter(filter_order, normalized_cutoff, btype='low')
        samples = samples.astype(np.float64)  # Convert samples to float64
        samples = lfilter(b, a, samples)

    # Quantize samples based on the specified format and depth
    if quantization_format == 'int':
        if sample_depth == 16:
            samples = np.int16(samples * 32767)
        elif sample_depth == 24:
            samples = np.int32(samples * 8388607)
        elif sample_depth == 32:
            samples = np.int32(samples * 2147483647)
        else:
            raise ValueError("Unsupported sample depth for int format. Choose from 16, 24, 32.")
    elif quantization_format == 'float':
        if sample_depth == 32:
            samples = np.float32(samples)
        elif sample_depth == 64:
            samples = np.float64(samples)
        else:
            raise ValueError("Unsupported sample depth for float format. Choose from 32, 64.")
    else:
        raise ValueError("Unsupported quantization format. Choose 'int' or 'float'.")

    # Save the audio file
    write(filename, sampling_rate, samples)
    print(f"Generated white noise audio file with low-pass filter: {filename}")

In [4]:
def generate_linear_scan(duration=1, sampling_rate=44100, start=0.01, end=0.99, filename='scan.txt'):
    samples = np.linspace(start, end, num=sampling_rate * duration)
    samples = np.round(samples * 100) / 100

    with open(filename, 'w') as f:
        for i, s in enumerate(samples):
            f.write(f"{i/sampling_rate} {s}\n")
    print(f"Generated linear scan data file: {filename}")


def generate_sin_scan(duration=1, sampling_rate=44100, frequency=5, amplitude_min=0.01, amplitude_max=0.99, filename='scan.txt'):
    samples = np.sin(2 * np.pi * frequency * np.linspace(0, duration, num=sampling_rate * duration))
    samples = (samples + 1) / 2 * (amplitude_max - amplitude_min) + amplitude_min
    samples = np.round(samples * 100) / 100

    with open(filename, 'w') as f:
        for i, s in enumerate(samples):
            f.write(f"{i/sampling_rate} {s}\n")
    print(f"Generated sin scan data file: {filename}")

def generate_triangle_scan(duration=1, sampling_rate=44100, frequency=5, amplitude_min=0.01, amplitude_max=0.99, filename='triangle_scan.txt'):
    time = np.linspace(0, duration, num=sampling_rate * duration)
    samples = np.abs(2 * (time * frequency % 1) - 1)
    samples = samples * (amplitude_max - amplitude_min) + amplitude_min
    samples = np.round(samples * 100) / 100
    
    with open(filename, 'w') as f:
        for i, s in enumerate(samples):
            f.write(f"{i/sampling_rate} {s}\n")
    
    print(f"Generated triangle scan data file: {filename}")

In [5]:
duration = 60
sampling_rate = 192000
sample_depth = 32
quantization_format = 'int'
cutoff_freq = None
order = 5
filename = '../../ltspice/x1.wav'
generate_white_noise(duration, sampling_rate, sample_depth, quantization_format, filename, cutoff_freq)

Generated white noise audio file with low-pass filter: ../../ltspice/x1.wav


In [6]:
duration = 5
sampling_rate = 192000
filename = '../../ltspice/scan1.txt'
generate_linear_scan(duration, sampling_rate, 0.01, 0.99, filename)
# generate_triangle_scan(duration, sampling_rate, 5, 0.01, 0.99, filename)

Generated linear scan data file: ../../ltspice/scan1.txt


In [7]:
duration = 5
sampling_rate = 192000
filename = '../../ltspice/scan2.txt'
generate_triangle_scan(duration, sampling_rate, 1, 0.01, 0.99, filename)

Generated triangle scan data file: ../../ltspice/scan2.txt
